# Get Disciplines of the Editors

## Load data

In [1]:
import pandas as pd
import ast
from tqdm import tqdm
import numpy as np

In [2]:
with open('../../Data/Plosone(allofplos)/plosone_editor.txt') as f:
    lines = f.readlines()

In [3]:
editor_sections=[]
editor = []
for i,line in enumerate(lines):
    line = line.replace('\n',"")
    editor.append(line)
    if 'Keywords:' in line:
        editor_sections.append(editor)
        editor = []
        

In [4]:
editor_sections[0]

['Katriina Aalto-Setala',
 'University of Tampere',
 'FINLAND',
 'Sections: Developmental biology - Cell differentiation; Cell fate determination; Stem cells; Embryology; Fertilization',
 'Keywords: Biology and life sciences, Cellular types, Stem cells, Induced pluripotent stem cells, Embryonic stem cells, Developmental biology, Cell differentiation, Genetics, Genetics of disease, Genetic testing, Human genetics, Genetic association studies, Molecular genetics, Cardiovascular anatomy, Electrophysiology, Medicine and health sciences, Cardiology, Arrhythmia, Clinical genetics, Personalized medicine, Animal models, Mouse models, Model organisms']

## Discipline Hierarchy

you can get the original data from [here](https://github.com/PLOS/plos-thesaurus)

In [5]:
section_with_comma = ['Computer and information sciences - Artificial intelligence, machine learning and data science','Engineering and technology - Aerospace and automotive engineering, Robotics'
                     ,'Public health and epidemiology - Health policies, systems and management','Public health and epidemiology - Health behavior, health promotion and society',
                     'Cancer - Biomarkers, molecular diagnostics and screening','Earth sciences - Geology, geochemistry and petrology'
                     ,'Ear, nose and throat (ENT)','Physics and astronomy - Nuclear, particle and plasma physics']

In [6]:
full=pd.read_excel('/home/balla/munjung/plos-thesaurus/plosthes.2020-1.full.xlsx')

In [7]:
full

,Tier 1,Tier 2,Tier 3,Tier 4,Tier 5,Tier 6,Tier 7,Tier 8,Tier 9,Tier 10,Tier 11,Unnamed: 11
0,Biology and life sciences,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,Agriculture,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,Agricultural biotechnology,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,Genetically modified foods,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,Marker-assisted selection,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
18373,NaN,NaN,Social theory,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18374,NaN,NaN,Social welfare,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18375,NaN,NaN,NaN,Welfare (social security),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18376,NaN,NaN,Sociology of knowledge,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
dis_index=full[full['Tier 1'].notna()].index

subject_hierarchy = {}
for i,ind in enumerate(dis_index):
    if i == 10:
        all_sub = full.loc[ind:].values.tolist()
    else:
        all_sub = full.loc[ind:dis_index[i+1]-1].values.tolist()
    subject_hierarchy[full['Tier 1'][ind]] = [sub for sublist in all_sub for sub in sublist if str(sub) != 'nan']

## Get tier 1 disciplines in keywords of each editor information

In [9]:
editor_name_section = []
for editor_info in tqdm(editor_sections):
    
    university = editor_info[1]
    keywords_list = []
    
    for info in editor_info:
        if 'orcid' in info:
            university = editor_info[2]
            
        if 'Keywords:' in info:
            
            
            info = info.replace("Keywords: ","")
            Keywords = info.split(', ')
            
            for keyword in Keywords:
                if keyword in list(set(full[full['Tier 1'].notna()]['Tier 1'])):
                    keywords_list.append(keyword)
                    
                    
            
         
                    
        
        if 'Sections:' in info:
            section=[]
            
            info = info.replace("Sections: ","")
            info = info.replace("Sections:","")
            
            for comma in section_with_comma:
                if comma in info:
                    section.append(comma)
                    info = info.replace(comma,"")
            
            
            info_split = info.split(', ')
            
            for i in info_split:
                if (i!=' ') and i!='':
                    if '-' not in i: 
                        if i[0].islower():
                            print(info_split)
                    section.append(i)
                    
                    

    editor = {"Editor": [editor_info[0],[university]],"Section": section,"Tier 1":keywords_list}

    del keywords_list
    del Keywords
    del section
    del university
    editor_name_section.append(editor)

    

100%|██████████| 9548/9548 [01:51<00:00, 85.81it/s] 


In [10]:
editor_name_section = pd.DataFrame(editor_name_section)

In [11]:
editor_name_section 

,Editor,Section,Tier 1
0,"[Katriina Aalto-Setala, [University of Tampere]]",[Developmental biology - Cell differentiation;...,"[Biology and life sciences, Medicine and healt..."
1,"[Nurul Hidayah Ab Rahman, [Universiti Tun Huss...",[Computer and information sciences - Cryptogra...,[Computer and information sciences]
2,"[Mohd Nadhir Ab Wahab, [Universiti Sains Malay...",[Computer and information sciences - Artificia...,"[Computer and information sciences, Engineerin..."
3,"[Ukachukwu Okoroafor Abaraogu, [Glasgow Caledo...","[Physiotherapy, Health care - Health services ...",[Medicine and health sciences]
4,"[Behnam Abasht, [University of Delaware]]","[Agriculture - Animals, Genetics - Gene functi...",[Biology and life sciences]
...,...,...,...
9543,"[Vahit Özmen, [Istanbul University Istanbul Fa...","[Surgery - General, Cancer - Chemotherapy and ...",[Medicine and health sciences]
9544,"[Selcen Öztürkcan, [Linnaeus University (prima...","[Management science, Computer and information ...","[Social sciences, Engineering and technology, ..."
9545,"[Branislav T. Šiler, [Institute for Biological...","[Plant biology, Food science and technology, G...",[Biology and life sciences]
9546,"[Lovro Šubelj, [University of Ljubljana]]",[Computer and information sciences - Artificia...,[Computer and information sciences]


In [12]:
editor_name_section['Editor'] = editor_name_section['Editor'].apply(lambda x: str(x))
editor_name_section['Section'] = editor_name_section['Section'].apply(lambda x: str(x))
editor_name_section['Tier 1'] = editor_name_section['Tier 1'].apply(lambda x: str(x))

In [13]:
editor_name_section = editor_name_section.drop_duplicates()

In [14]:
editor_name_section[editor_name_section['Tier 1']=='[]']

,Editor,Section,Tier 1
10,"['Patricia A. Abbott', ['University of Michiga...","['Nursing', 'Biotechnology - General']",[]
17,"['Amar Abderrahmani', ['Centre National de la ...",['Cell biology - Cell membranes; Vesicles; Cel...,[]
44,"['Angel Abuelo', ['Michigan State University']]","['Agriculture - Animals', 'Immunology - Immuni...",[]
52,"['Bipul R. Acharya', ['University of Virginia ...",['Cell biology - Cytoskeleton; Cell adhesion; ...,[]
71,"['Zach N Adelman', ['Texas A&M University Coll...",['Genetics - General'],[]
...,...,...,...
9489,"['Fang Zhu', ['Pennsylvania State University']]","['Entomology', 'Genetics - Population genetics...",[]
9492,"['Liping Zhu', ['Institute of Tibetan Plateau ...","['Earth sciences - Multidisciplinary', 'Earth ...",[]
9494,"['Rong Zhu', ['Flinders University']]",['Economics - Health economics'],[]
9496,"['Wei Zhu', ['Stony Brook University (State Un...",['Computer and information sciences - Artifici...,[]


In [15]:
def get_tier1(x):
    sectionlist = x['Section']
    
    sectionlist = ast.literal_eval(sectionlist)
    
    keywords = ast.literal_eval(x['Tier 1'])
    if len(keywords)!=0:
        return keywords
    
    section_return = []
    for section in sectionlist:
        if '-' in section:
            section_split = section.split(' - ')
            if section_split[0] in list(set(full[full['Tier 1'].notna()]['Tier 1'])):
                section_return.append(section_split[0])
    if len(section_return)!=0:
        return section_return
    else:
        return '[]'

In [16]:
editor_name_section['Tier 1'] =  editor_name_section.apply(lambda x: get_tier1(x),axis=1)

/home/balla/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [17]:
editor_name_section[editor_name_section['Tier 1']=='[]']

,Editor,Section,Tier 1
10,"['Patricia A. Abbott', ['University of Michiga...","['Nursing', 'Biotechnology - General']",[]
17,"['Amar Abderrahmani', ['Centre National de la ...",['Cell biology - Cell membranes; Vesicles; Cel...,[]
44,"['Angel Abuelo', ['Michigan State University']]","['Agriculture - Animals', 'Immunology - Immuni...",[]
52,"['Bipul R. Acharya', ['University of Virginia ...",['Cell biology - Cytoskeleton; Cell adhesion; ...,[]
71,"['Zach N Adelman', ['Texas A&M University Coll...",['Genetics - General'],[]
...,...,...,...
9477,"['Renping Zhou', ['Rutgers University']]",['Neuroscience - Cellular and molecular'],[]
9485,"['Yifeng Zhou', ['Soochow University Medical C...",['Cancer - Basic cancer research'],[]
9489,"['Fang Zhu', ['Pennsylvania State University']]","['Entomology', 'Genetics - Population genetics...",[]
9494,"['Rong Zhu', ['Flinders University']]",['Economics - Health economics'],[]


In [18]:
editor_name_section['Editor'] = editor_name_section['Editor'].apply(lambda x: str(x))
editor_name_section['Section'] = editor_name_section['Section'].apply(lambda x: str(x))
editor_name_section['Tier 1'] = editor_name_section['Tier 1'].apply(lambda x: str(x))

/home/balla/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/home/balla/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/home/balla/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instea

In [19]:
editor_name_section['index'] = editor_name_section.index

/home/balla/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [20]:
def get_tier1_section(x):
    sectionlist = x['Section']
    
    sectionlist = ast.literal_eval(sectionlist)
    
    Tier1 = ast.literal_eval(x['Tier 1'])
    
    if len(Tier1)!=0:
        return Tier1

    
    
    section_return = []
    for section in sectionlist:
        if '-' in section:
            section_split = section.split(' - ')[0]
        else:
            section_split = section
            
        if section_split in np.array(full):
            index = np.where(np.array(full) ==section_split)[0][0]
            for j,i in enumerate(full[full['Tier 1'].notna()]['Tier 1'].index):
                if (full[full['Tier 1'].notna()]['Tier 1'].index[j]<index):
                    if j!= len(full[full['Tier 1'].notna()]['Tier 1'].index)-1:
                        if (full[full['Tier 1'].notna()]['Tier 1'].index[j+1]>index):
                            section_return.append(full[full['Tier 1'].notna()]['Tier 1'][i])
                    else:
                         section_return.append(full[full['Tier 1'].notna()]['Tier 1'][i])
    
    
    if len(section_return)!=0:
        return [most_common(section_return)]
    else:
        return '[]'

In [21]:
def most_common(lst):
    return max(set(lst), key=lst.count)

In [22]:
editor_name_section

,Editor,Section,Tier 1,index
0,"['Katriina Aalto-Setala', ['University of Tamp...",['Developmental biology - Cell differentiation...,"['Biology and life sciences', 'Medicine and he...",0
1,"['Nurul Hidayah Ab Rahman', ['Universiti Tun H...",['Computer and information sciences - Cryptogr...,['Computer and information sciences'],1
2,"['Mohd Nadhir Ab Wahab', ['Universiti Sains Ma...",['Computer and information sciences - Artifici...,"['Computer and information sciences', 'Enginee...",2
3,"['Ukachukwu Okoroafor Abaraogu', ['Glasgow Cal...","['Physiotherapy', 'Health care - Health servic...",['Medicine and health sciences'],3
4,"['Behnam Abasht', ['University of Delaware']]","['Agriculture - Animals', 'Genetics - Gene fun...",['Biology and life sciences'],4
...,...,...,...,...
9543,"['Vahit Özmen', ['Istanbul University Istanbul...","['Surgery - General', 'Cancer - Chemotherapy a...",['Medicine and health sciences'],9543
9544,"['Selcen Öztürkcan', ['Linnaeus University (pr...","['Management science', 'Computer and informati...","['Social sciences', 'Engineering and technolog...",9544
9545,"['Branislav T. Šiler', ['Institute for Biologi...","['Plant biology', 'Food science and technology...",['Biology and life sciences'],9545
9546,"['Lovro Šubelj', ['University of Ljubljana']]",['Computer and information sciences - Artifici...,['Computer and information sciences'],9546


In [23]:
editor_name_section['Tier 1'] =  editor_name_section.apply(lambda x: get_tier1_section(x),axis=1)

/home/balla/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [24]:
editor_name_section['Editor'] = editor_name_section['Editor'].apply(lambda x: str(x))
editor_name_section['Section'] = editor_name_section['Section'].apply(lambda x: str(x))
editor_name_section['Tier 1'] = editor_name_section['Tier 1'].apply(lambda x: str(x))

/home/balla/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/home/balla/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/home/balla/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instea

In [25]:
editor_name_section[editor_name_section['Tier 1']=='[]']

,Editor,Section,Tier 1,index
94,"['Pierfrancesco Agostoni', ['Antonius Ziekenhu...",['Cardiovascular science and medicine - Corona...,[],94
128,"['Shawn Ahmed', ['University of North Carolina...",['Cancer - Basic cancer research'],[],128
176,"['Nehad M Alajez', ['Qatar Biomedical Research...",['Cancer - Basic cancer research'],[],176
252,"['Eduardo G. Altmann', ['University of Sydney']]",['Physics and astronomy - Multidisciplinary'],[],252
273,"['Roberto Amendola', ['ENEA Centro Ricerche Ca...",['Cancer - Basic cancer research'],[],273
...,...,...,...,...
9224,"['Marcel Yotebieng', ['Albert Einstein College...",['Public health and epidemiology - Health beha...,[],9224
9305,"['RunGuo Zang', ['Chinese Academy of Forestry']]",['Applied ecology and resource management - Fo...,[],9305
9347,"['Aimin Zhang', ['Institute of Genetics and De...","['Plant biology', 'Applied ecology and resourc...",[],9347
9485,"['Yifeng Zhou', ['Soochow University Medical C...",['Cancer - Basic cancer research'],[],9485


## Section names which are not in the the thesaurus list

In [26]:
hierarchy = {"Ecology and environmental sciences" : ['Plant biology','Applied ecology and resource management'],
 "Biology and life sciences" :['Plant biology','Structural biology','Cell biology'],
 "Medicine and health sciences":['Ear, nose and throat (ENT)','Nursing','Pathology','Sport and exercise science',"Women's and maternal health",'Cancer','Public health and epidemiology','Cardiovascular science and medicine','Surgery','Bone research and musculoskeletal disorders','Dentistry and oral health'],
"Physical sciences": ['Physics and astronomy'],
"Social sciences" : ['Psychology','Economics','Social networks',],
    "Computer and information sciences" : ['Complexity and networks']
           }


In [27]:
for a in [x for x in hierarchy.keys()]:
    print(a)
    print(hierarchy[a])
    print("==========================")

Ecology and environmental sciences
['Plant biology', 'Applied ecology and resource management']
Biology and life sciences
['Plant biology', 'Structural biology', 'Cell biology']
Medicine and health sciences
['Ear, nose and throat (ENT)', 'Nursing', 'Pathology', 'Sport and exercise science', "Women's and maternal health", 'Cancer', 'Public health and epidemiology', 'Cardiovascular science and medicine', 'Surgery', 'Bone research and musculoskeletal disorders', 'Dentistry and oral health']
Physical sciences
['Physics and astronomy']
Social sciences
['Psychology', 'Economics', 'Social networks']
Computer and information sciences
['Complexity and networks']


In [28]:
def get_tier1_section_full(x):
    sectionlist = x['Section']
    
    
    
    sectionlist = ast.literal_eval(sectionlist)
    
    keywords = ast.literal_eval(x['Tier 1'])

    
    
    
    if len(keywords)!=0:
        return keywords
    
    section_return = []
    for section in sectionlist:
        if '-' in section:
            section_split = section.split(' - ')[0]
        else:
            section_split = section
            
        for i,topsub in enumerate([x for x in hierarchy.values()]):
            if section_split in topsub:
                section_return.append([x for x in hierarchy.keys()][i])
    if len(section_return)!=0:
        return [most_common(section_return)]
    else:
        return '[]'

In [29]:
editor_name_section['Tier 1'] =  editor_name_section.apply(lambda x: get_tier1_section_full(x),axis=1)

/home/balla/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [30]:
editor_name_section[editor_name_section['Tier 1']=='[]']

,Editor,Section,Tier 1,index
3424,"['Sander L. Hitzig', ['Sunnybrook Research Ins...",[],[],3424
4399,"['Nirbhay Kumar', ['George Washington Universi...",[],[],4399
6929,"['Eric Roessner', ['University Medical Center ...",[],[],6929


In [31]:
editor_name_section['Editor'] = editor_name_section['Editor'].apply(lambda x: str(x))
editor_name_section['Section'] = editor_name_section['Section'].apply(lambda x: str(x))
editor_name_section['Tier 1'] = editor_name_section['Tier 1'].apply(lambda x: str(x))

/home/balla/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/home/balla/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/home/balla/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instea

In [36]:
ls = [1,2,3,4]

In [37]:
for i,j in enumerate(ls):
    print(i,j)

0 1
1 2
2 3
3 4


## No section data

In [38]:
def get_tier1_keywords_full(x):
    sectionlist = x['Section']
    
    
    
    sectionlist = ast.literal_eval(sectionlist)
    
    tier1 = ast.literal_eval(x['Tier 1'])
    
    
    
    
    if len(tier1)!=0:
        return tier1
    
    ind = x['index']
    
    editor_info = editor_sections[ind]

    
    keywords_list = []
    
    for info in editor_info:
        if 'Keywords: ' in info:
           
            info = info.replace("Keywords: ","")
            info = info.split(', ')
            
            for keyword in info:
                if keyword in np.array(full):
                    print(keyword)
                    index = np.where(np.array(full) ==keyword)[0][0]
                    for j,i in enumerate(full[full['Tier 1'].notna()]['Tier 1'].index):
                        if (full[full['Tier 1'].notna()]['Tier 1'].index[j]<index):
                            if j!= len(full[full['Tier 1'].notna()]['Tier 1'].index)-1:
                                if (full[full['Tier 1'].notna()]['Tier 1'].index[j+1]>index):
                                    keywords_list.append(full[full['Tier 1'].notna()]['Tier 1'][i])
                            else:
                                 keywords_list.append(full[full['Tier 1'].notna()]['Tier 1'][i])
    print(keywords_list)
    print("======================")                
    if len(keywords_list)!=0:
        return [most_common(keywords_list)]
    else:
        return '[]'

In [39]:
editor_name_section['Keywords'] =  editor_name_section.apply(lambda x: get_tier1_keywords_full(x),axis=1)

Social networks
Quality of life
Dementia
Spinal cord injury
Rehabilitation medicine
Functional electrical stimulation
Surgical amputation
Burns
Canada
Elderly
Qualitative studies
Surveys
['Computer and information sciences', 'Medicine and health sciences', 'Medicine and health sciences', 'Medicine and health sciences', 'Medicine and health sciences', 'Medicine and health sciences', 'Medicine and health sciences', 'Medicine and health sciences', 'People and places', 'People and places', 'Research and analysis methods', 'Research and analysis methods']
Gametocytes
Malarial parasites
Oocysts
Plasmodium
Blood
Antibodies
Mosquitoes
Parasitic diseases
Malaria
DNA vaccination
Enzyme-linked immunoassays
Polymerase chain reaction
['Biology and life sciences', 'Biology and life sciences', 'Biology and life sciences', 'Biology and life sciences', 'Biology and life sciences', 'Biology and life sciences', 'Biology and life sciences', 'Medicine and health sciences', 'Medicine and health sciences', '

/home/balla/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [40]:
editor_name_section[editor_name_section['Editor'].apply(lambda x: True if 'Nirbhay Kumar' in x else False)]

,Editor,Section,Tier 1,index,Keywords
4399,"['Nirbhay Kumar', ['George Washington Universi...",[],[],4399,[Biology and life sciences]


In [42]:
editor_name_section[editor_name_section['Keywords']=='[]']

,Editor,Section,Tier 1,index,Keywords


In [43]:
editor_name_section.to_csv('../../Data/Plosone(allofplos)/editor_name_section.csv')